In [ ]:
!pip install mido
!pip install pygame
!pip install music21
!pip install scikit-learn==1.3.0

In [1]:
# import some useful libraries
import glob, nltk, joblib
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import metrics

from PIL import Image
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer

from music21 import midi
from plugins.midi2img import midi2img
from plugins.img2midi import img2midi
from IPython.display import clear_output

# Load In museGAN dataset for visualization purposes
It turned out that the people at museGAN is leveraging midi -> image conversion. The image consisted of bar of a multi track piano roll. From the below image, the horizontal represent time and the vericle represent the instrument used. In this dataset the instrument are layered from bottom to top as piano, strings, guitar, drums, bass.


In [2]:
# download the punkt tokenizer from nltk to tokenize the piece caption
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ktrin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Construction of museGAN
## External Data Source
For whatever reason, if we wanted to perform GAN modeling, we can leverage conversion of MIDI data to that of the piano roll. Download the data from piano repo in README and start performing the things below. [Convert-MIDI-TO-NP-ARRAY](https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c)

# Caption Processing

In [3]:
MAX_SEQ_LENGTH = 20 # 18 + start, end
EMBED_DIM = 100 
MAX_VOCAB_SIZE = 20000

In [4]:
# load in the metadata
# create a list of captions that concatenate the piece description and arousal
# lower case te caption list
midi_meta = pd.read_csv('../data/piano-labelled/labelled_piano_midi_metadata.csv')
midi_meta['caption_list'] = midi_meta['piece_description'].str.lower()+ ". " + midi_meta['piece_arousal'].str.lower()
midi_meta

,piece_id,piece_description,piece_arousal,piece_name,midi_file,caption_list
0,0,very upbeat,Delighted,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,very upbeat. delighted
1,1,I could tell the valence of the example was in...,Valence started out moderately negative and pr...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,i could tell the valence of the example was in...
2,2,For a second I thought this piece was going to...,This piece seemed to have a positive valence t...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,for a second i thought this piece was going to...
3,3,Bouncy and fun,Kind of sparatic,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,bouncy and fun. kind of sparatic
4,4,nice,nice,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,nice. nice
...,...,...,...,...,...,...
6100,6100,It started off slowly but happy and then built...,Seemed to remain consistent almost like it was...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,it started off slowly but happy and then built...
6101,6101,This starts off a certain way then changes in ...,This is nostalgic because I recognize this and...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,this starts off a certain way then changes in ...
6102,6102,The piece begins slow in tempo and then become...,"The beginning rhythm sounds suspenseful, makin...",One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,the piece begins slow in tempo and then become...
6103,6103,started slow but picked up.,I feel it stayed the same,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,started slow but picked up. . i feel it stayed...


In [5]:
# build a vocabulary using sklearn count vectorizer to create a vocab from the most frequent words
input_captions = []
max_caption_length = -1 

for caption in tqdm(midi_meta['caption_list'].values):
    tokenized_caption = nltk.word_tokenize(caption, language='english')

    if len(tokenized_caption) > max_caption_length:
        max_caption_length = len(tokenized_caption)

    caption = (' '.join(tokenized_caption)).lower()
    input_captions.append(caption)


vectorizer = CountVectorizer(max_features=MAX_VOCAB_SIZE)
vectorizer.fit(input_captions)
vocab = vectorizer.get_feature_names_out()
MAX_VOCAB_SIZE = len(vocab)

  0%|          | 0/6105 [00:00<?, ?it/s]

In [6]:
# turn vocab into a dictionary of words and token id
# replace some words with special tokens like start/end/unk
# if the caption is too short, pad it with <pad> token
id_vocab_dict = {}
vocab_id_dict = {}

for sid, svocab in enumerate(vocab):
    id_vocab_dict[sid] = svocab
    vocab_id_dict[svocab] = sid

id_vocab_dict[MAX_VOCAB_SIZE] = "<unk>"
id_vocab_dict[MAX_VOCAB_SIZE + 1] = "<start>"
id_vocab_dict[MAX_VOCAB_SIZE + 2] = "<end>"
id_vocab_dict[MAX_VOCAB_SIZE + 3] = "<pad>"

vocab_id_dict["<unk>"] = MAX_VOCAB_SIZE
vocab_id_dict["<start>"] = MAX_VOCAB_SIZE + 1
vocab_id_dict["<end>"] = MAX_VOCAB_SIZE + 2
vocab_id_dict["<pad>"] = MAX_VOCAB_SIZE + 3

In [7]:
# tokenization - take the input caption and tokenize it
# declare a max sequence length 
def convert_text_to_data(texts, 
                         vocab_id_dict, 
                         max_length=20, 
                         type=None):
    """
        Function to convert text based data into tokenized data with proper padding
    """

    processed_data = []
    for text_num, text in enumerate(texts):
        sentence_ids = []

        # split the sentence into token
        # use the vocab to turn the word token into number
        for token in text.split():
            if token in vocab_id_dict.keys():
                sentence_ids.append(vocab_id_dict[token])
            else:
                sentence_ids.append(vocab_id_dict["<unk>"])

        vocab_size = len(vocab_id_dict.keys())

        # for decoder cases:
        # input sentence: <start>, [tokenize words from vocab], <end>, padded with <unk>
        # ouput sentence has: [tokenize words from vocab], <end>, padded with <unk>
        if type == 'input_target':
            ids = ([vocab_size - 3] + sentence_ids + [vocab_size - 2] + [vocab_size - 1] * max_length)[:max_length]
        elif type == 'output_target':
            ids = (sentence_ids + [vocab_size - 2] + [vocab_size - 1] * max_length)[:max_length]
        processed_data.append(ids)

    return np.array(processed_data)


train_target_input_data = convert_text_to_data(input_captions,
                                                vocab_id_dict,
                                                type='input_target',
                                                max_length=MAX_SEQ_LENGTH)
len(train_target_input_data)

6105

In [8]:
# added the tokenized caption to the metadata
midi_meta['tokenized_captions'] = train_target_input_data.tolist()
midi_meta

,piece_id,piece_description,piece_arousal,piece_name,midi_file,caption_list,tokenized_captions
0,0,very upbeat,Delighted,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,very upbeat. delighted,"[3452, 3319, 3258, 3451, 762, 3453, 3454, 3454..."
1,1,I could tell the valence of the example was in...,Valence started out moderately negative and pr...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,i could tell the valence of the example was in...,"[3452, 3451, 661, 3025, 3053, 3290, 2095, 3053..."
2,2,For a second I thought this piece was going to...,This piece seemed to have a positive valence t...,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,for a second i thought this piece was going to...,"[3452, 1220, 3451, 2627, 3451, 3078, 3074, 224..."
3,3,Bouncy and fun,Kind of sparatic,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,bouncy and fun. kind of sparatic,"[3452, 364, 138, 1262, 3451, 1686, 2095, 2820,..."
4,4,nice,nice,Lurking In The Darkness,Final Fantasy_PS1_Final Fantasy VII_Lurking In...,nice. nice,"[3452, 2044, 3451, 2044, 3453, 3454, 3454, 345..."
...,...,...,...,...,...,...,...
6100,6100,It started off slowly but happy and then built...,Seemed to remain consistent almost like it was...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,it started off slowly but happy and then built...,"[3452, 1623, 2869, 2096, 2756, 413, 1374, 138,..."
6101,6101,This starts off a certain way then changes in ...,This is nostalgic because I recognize this and...,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,this starts off a certain way then changes in ...,"[3452, 3074, 2872, 2096, 3451, 468, 3366, 3061..."
6102,6102,The piece begins slow in tempo and then become...,"The beginning rhythm sounds suspenseful, makin...",One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,the piece begins slow in tempo and then become...,"[3452, 3053, 2241, 304, 2751, 1510, 3031, 138,..."
6103,6103,started slow but picked up.,I feel it stayed the same,One Winged Angel,Final Fantasy_PS1_Final Fantasy VII_One Winged...,started slow but picked up. . i feel it stayed...,"[3452, 2869, 2751, 413, 2233, 3451, 3451, 3451..."


# WGAN Definition

Enhancement to the original GAN model where the Wasserstein distance is leverage to produce a value function that has better theoretical properties than orginal GAN.
GAN model consists of two part:
1. Generator
2. Discriminator

In [9]:
# check to see if tensorflow mount to GPU properly
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
def caption_enhanced_generator(latent_dim=100, 
                               caption_dim=MAX_SEQ_LENGTH, 
                               vocab_size=len(vocab_id_dict.keys()), 
                               embed_dim=EMBED_DIM):
    """Define the generator model
        Inputs:
            latent_dim: dimension of the latent space
        Output:
            model: the generator model
    """
    n_nodes = 128 * 53 * 53

    # vectorized input layers
    input_layer = keras.layers.Input(shape=(latent_dim,), name='input_layer')
    
    # # vectorized caption input layers
    # # apply word embedding to the caption
    caption_input_layer = keras.layers.Input(shape=(caption_dim,), name='caption_input_layer')
    embedding_layer  = keras.layers.Embedding(input_dim=vocab_size,
                                                output_dim=embed_dim,
                                                name='caption_embedding_layer')
    embed_caption = embedding_layer(caption_input_layer)

    # # source_image_encoding = keras.layers.GlobalAveragePooling2D()(dense4)
    # # using LSTM to encode the caption with the input layer
    lstm_layer = keras.layers.LSTM(100, return_sequences=True, return_state=True, name="decoder_lstm_layer")
    decoder_output, decoder_state_h_output, decoder_state_c_output = lstm_layer(embed_caption, initial_state=[input_layer, input_layer])

    # apply 1D Global Average Pooling to the output of the dense layer on the caption decoded
    # global_average_pooling1d_layer = keras.layers.GlobalAveragePooling1D()(decoder_output)

    # Dense Layer 1
    dense1 = keras.layers.Dense(n_nodes, activation='relu')(decoder_state_c_output)
    leaky_relu1 = keras.layers.LeakyReLU(alpha=0.3)(dense1)
    reshape_layer = keras.layers.Reshape((53, 53, 128))(leaky_relu1)

    # Dense Layer 2
    dense2 =  keras.layers.Dense(1024)(reshape_layer)

    # Conv2DTranspose Layer
    conv2d_transpose = keras.layers.Conv2DTranspose(1024, (4, 4), strides=(2, 2), padding='same')(dense2)

    # Dense Layer 3
    dense3 =  keras.layers.Dense(1024)(conv2d_transpose)
    leaky_relu2 = keras.layers.LeakyReLU(alpha=0.3)(dense3)

    # Dense Layer 4
    dense4 =  keras.layers.Dense(1024)(leaky_relu2)

    # Conv2D Layer
    conv2d = keras.layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')(dense4)


    # Create the model
    model = keras.Model(inputs=[input_layer,caption_input_layer], outputs=conv2d, name='generator')
    return model


In [11]:
def conv_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = keras.layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = keras.layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = keras.layers.Dropout(drop_value)(x)
    return x

def caption_enhanced_discriminator(in_shape = (106,106,1)):
    """
        GAN discriminator model
        Inputs:
            in_shape: shape of the input image
        Output:
            model: discriminator model with binary crossentropy loss to denotes if the image is real or fake
    """
    # Input Layer
    input_layer = keras.layers.Input(shape=in_shape, name='input_layer')
    
     
    # 2D Convlution Layer 1
    conv1 = keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same')(input_layer)
    leaky_relu1 = keras.layers.LeakyReLU(alpha=0.2)(conv1)
    dropout1 = keras.layers.Dropout(0.5)(leaky_relu1)

    # 2D Convlution Layer 2
    conv2 = keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same')(dropout1)
    leaky_relu2 = keras.layers.LeakyReLU(alpha=0.2)(conv2)
    dropout2 = keras.layers.Dropout(0.5)(leaky_relu2)

    # Flatten Layer
    flatten_layer = keras.layers.Flatten()(dropout2)

    # Batch Normalization Layer
    batch_normalization = keras.layers.BatchNormalization()(flatten_layer)
    
    # Dense Output Disminator Layer
    discriminate_layer = keras.layers.Dense(1)(batch_normalization)

    # Create the model
    model = keras.Model(inputs=input_layer, outputs=discriminate_layer, name='discriminator')

    return model 

In [12]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn



    def gradient_penalty(self, batch_size, real_images, fake_images):
        """Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image by adding a random noise the the residuals
        alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff


        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated])[0]

        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp
    

    def train_step(self, dataset):
        # Unpack the data
        real_images, real_captions = dataset

        if isinstance(real_images, tuple):
            real_images = real_images[0]

        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):

            # Get the latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator([random_latent_vectors, real_captions], training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator(fake_images, training=True)
                # Get the logits for the real images
                real_logits = self.discriminator(real_images, training=True)

                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator([random_latent_vectors, real_captions], training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss": d_loss, "g_loss": g_loss}

In [13]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=6, latent_dim=128, filepath="."):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        # random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        # generated_images = self.model.generator(random_latent_vectors)
        # generated_images = (generated_images * 127.5) + 127.5

        # for i in range(self.num_img):
        #     img = generated_images[i].numpy()
        #     img = keras.utils.array_to_img(img)
        #     img.save(f"{self.filepath}/generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch))


        # save the model 
        self.model.generator.save(f"{self.filepath}/generator_{epoch}.h5".format(epoch=epoch))
        self.model.discriminator.save(f"{self.filepath}/discriminator_{epoch}.h5".format(epoch=epoch))

In [14]:
g_model = caption_enhanced_generator()
d_model = caption_enhanced_discriminator()
g_model.summary(), d_model.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input_layer (InputLayer [(None, 20)]         0                                            
__________________________________________________________________________________________________
caption_embedding_layer (Embedd (None, 20, 100)      345500      caption_input_layer[0][0]        
__________________________________________________________________________________________________
input_layer (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
decoder_lstm_layer (LSTM)       [(None, 20, 100), (N 80400       caption_embedding_layer[0][0]    
                                                                 input_layer[0][0]        

(None, None)

In [15]:
# read in the training data
import random
random.seed(5634)

training_set = joblib.load('../data/piano-labelled/training_set.joblib')
training_set = random.sample(training_set, 2048) #5 is the lenth of the sample

In [16]:
# unzip the dataset
images, captions = zip(*training_set)
images = np.array(images)
captions = np.array(captions)
type(images), images.shape

(numpy.ndarray, (2048, 106, 106, 1))

In [18]:
# Set some training parameters
BATCH_SIZE = 8
noise_dim = 100
epochs = 3


# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9,
    clipnorm=2.0
)

discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9,
    clipnorm=2.0
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)


# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(num_img=3, latent_dim=noise_dim, filepath='../model/wgan-callbacks')

# Get the wgan model
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=3,
)

# Compile the wgan model
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)

# Start training
wgan.fit(images, captions, batch_size=BATCH_SIZE, epochs=epochs)

Epoch 1/3
256/256 [==============================] - 946s 4s/step - d_loss: 10.0000 - g_loss: 5.3343e-09
Epoch 2/3
256/256 [==============================] - 949s 4s/step - d_loss: 10.0000 - g_loss: 5.0262e-09
Epoch 3/3
150/256 [================>.............] - ETA: 6:09 - d_loss: 10.0000 - g_loss: 3.0547e-09

KeyboardInterrupt: 

In [19]:
# save the model
g_model.save('../models/mini-gan/caption_piano_wgan_generator_model.h5')
d_model.save('../models/mini-gan/caption_piano_wgan_discriminator_model.h5')


In [ ]:
# Load model
# g_model = keras.models.load_model('../models/mini-gan/caption_piano_wgan_generator_model.h5')

In [20]:
# select a random row from the metadata to get the caption
row = midi_meta.sample(1, random_state=63)

# get a random image tokenize caption and actual caption
NLP_caption = row['caption_list'].values
caption = [np.array(a) for a in row['tokenized_captions'].values]
caption = np.array(caption)
NLP_caption, caption

(array(['it goes from forced to worried . determination'], dtype=object),
 array([[3452, 1623, 1307, 1253, 1223, 3122, 3423, 3451,  800, 3453, 3454,
         3454, 3454, 3454, 3454, 3454, 3454, 3454, 3454, 3454]]))

In [21]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


# declare a latent space
latent_dim = 100
latent_points = generate_latent_points(latent_dim, 1)
latent_points.shape, caption.shape

((1, 100), (1, 20))

In [22]:
model = g_model
X = g_model.predict([latent_points, caption])
array = np.array(X.reshape(106,106),dtype = np.uint8)
np.unique(array)

array([0], dtype=uint8)

In [23]:
array*=255
new_image = Image.fromarray(array,'L')
new_image = new_image.save(f'../data/midi_reconstruction/images/wgan_captioned_piece_test.png')

In [24]:
# reconvert MIDI images to MIDI files
image_path = "../data/midi_reconstruction/images/wgan_captioned_piece_test.png"
output_path = "../data/midi_reconstruction"

img2midi_obj = img2midi(image_path, output_path, resolution=0.25)
img2midi_obj.convert_to_midi()

In [26]:
# run to cell to play
# stop the cell and run sp.stop to stop the music
from music21 import midi, converter, instrument, note, chord

mf = midi.MidiFile()
mf.open(f"{output_path}/music.mid")
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
sp = midi.realtime.StreamPlayer(s)
sp.play()

In [ ]:
sp.stop